# PuLP で組み合わせ問題 (Set Partitioning Problem) を解く  

PuLP は Python で線形計画 (Linear Programing) 問題をモデル化し、ソルバーで解くためのツールです。  

- [coin-or/pulp: A python Linear Programming API](https://github.com/coin-or/pulp)  
- [Optimization with PuLP](https://coin-or.github.io/pulp/)  
- [pulp: Pulp classes](https://coin-or.github.io/pulp/technical/pulp.html)  

Anaconda Cloud ではパッケージは公開されていませんので、`pip install pulp` でインストールしてください。  

## 問題  

あるパーティの席割を検討しています。条件は、

- 参加者数は 18 名  
- テーブルは 5 つ  
- 各テーブルに参加者を 1 名以上、4 名以下を割り当てる  

です。この条件下で、各テーブルの幸福度の総和を最大化したいと考えています。幸福度は、    

- 参加者同士は 0 以上 1 未満の相性度というパラメータを持っている (大きいほど相性がよい)  
- 各テーブルに割り当てられた参加者同士の最も低い相性度をそのテーブルの幸福度とする

で求められるものとします。  

In [1]:
# 必要なモジュールのインポート
import pulp
from pulp import LpProblem, LpMaximize, LpVariable, \
                 LpInteger, lpSum
import numpy as np
import pandas as pd
import itertools

In [2]:
# 乱数のシード値を指定
np.random.seed(0)

In [3]:
max_tables = 5
max_table_size = 4
guests = "A B C D E F G H I J K L M N O P Q R".split()
num_guests = len(guests)
print("出席者数: {} 人".format(num_guests))

出席者数: 18 人


In [4]:
# 各個人間の相性度を作成する
# 対角成分が 0 で、それ以外は 0 以上 1 未満の値の対称行列とする
a = np.round(np.random.rand(num_guests**2), 4).reshape(num_guests, -1)
a_tri = np.triu(a)  # 上側に値を持つ三角行列
affinity = a_tri + a_tri.T - 2*np.diag(a_tri.diagonal())

In [5]:
pd.DataFrame(affinity, index=guests, columns=guests)

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R
A,0.0000,0.7152,0.6028,0.5449,0.4237,0.6459,0.4376,0.8918,0.9637,0.3834,0.7917,0.5289,0.5680,0.9256,0.0710,0.0871,0.0202,0.8326
B,0.7152,0.0000,0.9786,0.7992,0.4615,0.7805,0.1183,0.6399,0.1434,0.9447,0.5218,0.4147,0.2646,0.7742,0.4562,0.5684,0.0188,0.6176
C,0.6028,0.9786,0.0000,0.6818,0.3595,0.4370,0.6976,0.0602,0.6668,0.6706,0.2104,0.1289,0.3154,0.3637,0.5702,0.4386,0.9884,0.1020
D,0.5449,0.7992,0.6818,0.0000,0.4663,0.2444,0.1590,0.1104,0.6563,0.1382,0.1966,0.3687,0.8210,0.0971,0.8379,0.0961,0.9765,0.4687
E,0.4237,0.4615,0.3595,0.4663,0.0000,0.1202,0.2961,0.1187,0.3180,0.4143,0.0641,0.6925,0.5666,0.2654,0.5232,0.0939,0.5759,0.9293
F,0.6459,0.7805,0.4370,0.2444,0.1202,0.0000,0.5865,0.0201,0.8289,0.0047,0.6778,0.2700,0.7352,0.9622,0.2488,0.5762,0.5920,0.5723
G,0.4376,0.1183,0.6976,0.1590,0.2961,0.5865,0.0000,0.3965,0.8811,0.5813,0.8817,0.6925,0.7253,0.5013,0.9561,0.6440,0.4239,0.6064
H,0.8918,0.6399,0.0602,0.1104,0.1187,0.0201,0.3965,0.0000,0.5700,0.5909,0.5743,0.6532,0.6521,0.4314,0.8965,0.3676,0.4359,0.8919
I,0.9637,0.1434,0.6668,0.6563,0.3180,0.8289,0.8811,0.5700,0.0000,0.6156,0.1238,0.8480,0.8073,0.5691,0.4072,0.0692,0.6974,0.4535
J,0.3834,0.9447,0.6706,0.1382,0.4143,0.0047,0.5813,0.5909,0.6156,0.0000,0.2000,0.0185,0.7937,0.2239,0.3454,0.9281,0.7044,0.0318


In [6]:
# ひとつのテーブルに対して、起こりうるすべての席割のリストを作成
possible_tables = [tuple(c) for c in pulp.allcombinations(guests, max_table_size)]
print("{} 通りの組み合わせが存在".format(len(possible_tables)))

4047 通りの組み合わせが存在


In [7]:
# 最後の要素を確認
possible_tables[-1]

('O', 'P', 'Q', 'R')

In [8]:
# テーブルの幸福度を求める関数
def happiness(guests: list, table: tuple, aff: np.ndarray) -> float: 
    """
    テーブルの幸福度を割り当てられた人のリストと相性度テーブルから取得する関数。
    割り当てられた人から 2 人選んで最も低い相性度がそのテーブルの幸福度となる。
    """
    if len(table) <= 1: return 0  # 簡易な if 式の書き方
    
    ret = 1
    for c in itertools.combinations(table, 2):
        a = aff[guests.index(c[0]), guests.index(c[1])]
        if ret > a:
            ret = a            
    return ret

In [9]:
# 関数のテスト
happiness(guests, ("A", "B", "C"), affinity)

0.6028

In [10]:
# モデルの作成 目的関数を最大化する
zaseki = pulp.LpProblem(name="座席割", sense=pulp.LpMaximize)

In [11]:
# その席割を使用するかどうかの 2 値 (0 または 1) の決定変数を作成
x = pulp.LpVariable.dicts(name="table", indexs=possible_tables, 
                          lowBound=0, upBound=1, cat=pulp.LpInteger)
len(x)

4047

In [12]:
# 目的関数をモデルに追加
zaseki += lpSum([happiness(guests, table, affinity) * x[table] for table in possible_tables])

In [13]:
# 制約条件をモデルに追加

# テーブル数の制限
zaseki += lpSum([x[table] for table in possible_tables]) <= max_tables, "Maximum_number_of_tables"

# 1 人の出席者は必ずどれかひとつのテーブルに座らなければならない
for guest in guests:
    zaseki += lpSum([x[table] for table in possible_tables 
                     if guest in table]) == 1, "Must_seat_{}".format(guest)

In [14]:
zaseki.solve()

1

In [15]:
# 結果の確認
total_happiness = 0
for table in possible_tables:
    if pulp.value(x[table]) == 1.0:
        h = happiness(guests, table, affinity)
        print(table, h)
        total_happiness += h

print("-"*20, "\n幸福度の合計: {}".format(total_happiness))

('C', 'Q') 0.9884
('A', 'F', 'K', 'N') 0.6459
('B', 'H', 'J', 'P') 0.3676
('D', 'E', 'O', 'R') 0.4663
('G', 'I', 'L', 'M') 0.6925
-------------------- 
幸福度の合計: 3.1607


## 参考文献  

- [A Set Partitioning Problem — PuLP 2.0 documentation](https://coin-or.github.io/pulp/CaseStudies/a_set_partitioning_problem.html)  
- [問題解決のためのオペレーションズ・リサーチ入門｜日本評論社](https://www.nippyo.co.jp/shop/book/1404.html)  